# Levenberg-Marquardt

In [1]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as sopt
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt

In [2]:
def funcx(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(funcx([x[0]+delta,x[1]])- funcx([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(funcx([x[0],x[1]+delta])- funcx([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = funcx(x+alpha1*search);
    falpha2 = funcx(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = funcx(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = funcx(x+alpha1*search);

        if np.abs(funcx(x+alpha1*search)- funcx(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [5]:
def hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= ( funcx([x[0]+delta,x[1]])  - 2*funcx(x) + funcx([x[0]-delta,x[1]]) )/ delta**2;
    H[1,1]= ( funcx([x[0],x[1]+delta])  - 2*funcx(x) + funcx([x[0],x[1]-delta]) )/ delta**2; 
    H[0,1]= ( funcx([x[0]+delta,x[1]+delta]) - funcx([x[0]+delta,x[1]-delta]) - funcx([x[0]-delta,x[1]+delta]) + funcx([x[0]-delta,x[1]-delta]) )/ (4*(delta**2));
    H[1,0]=H[0,1]
    return H

In [7]:
delta=1e-3; 
ep1=1e-3; 
xi = [-1,1];
x = xi;
lam = float(1e3);
fx_prev=funcx(x)
print('Initial function value = {0:.3f} '.format(fx_prev))
print(' No.\tx-vector\tf(x)\tDeriv ')
print('------------------------------------------')
for i in range(30):
    fx_prev=funcx(x)
    dire=gradient(x,delta)
    H=hessian(x,delta)
    dire=np.atleast_2d(dire)
    si=np.matmul(-la.inv(H+lam*np.eye(len(x))),dire.transpose())
    x = x + si.transpose() 
    x=np.ndarray.flatten(x)
    fx_curr = funcx(x)
    if fx_curr < fx_prev:
        lam = lam/2
    else:
        lam = 2*lam

    if abs(fx_curr-fx_prev)<ep1 or la.norm(dire)<ep1:
        break;
    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i,x[0], x[1],fx_curr,la.norm(dire)))
print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i,x[0], x[1],fx_curr,la.norm(dire)))
print('------------------------------------------')

Initial function value = 270.294 
 No.	x-vector	f(x)	Deriv 
------------------------------------------
0	[-0.830,0.726]	146.756	444.316
1	[-0.625,0.420]	53.601	322.405
2	[-0.428,0.193]	14.428	184.483
3	[-0.232,0.114]	3.704	77.798
4	[0.026,0.128]	-2.509	32.798
5	[0.282,0.102]	-6.126	20.778
6	[0.426,0.043]	-7.192	12.723
7	[0.485,0.009]	-7.346	4.861
8	[0.498,0.001]	-7.353	1.013
9	[0.500,-0.000]	-7.353	0.085
------------------------------------------
